In [12]:
# ! pip install python-dotenv
# ! pip install langchain --upgrade --no-cache-dir
# ! pip install wandb --upgrade --no-cache-dir

In [3]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [4]:
%load_ext autoreload

In [5]:
%autoreload

import sys
import os
import json

# os.environ[ "LANGCHAIN_WANDB_TRACING" ] = "true"
# # wandb documentation to configure wandb using env variables
# # https://docs.wandb.ai/guides/track/advanced/environment-variables
# # here we are configuring the wandb project name
# os.environ[ "WANDB_PROJECT" ] = "deepily-dataframe-agent"

path = "/var/genie-in-the-box/src/lib"
if path not in sys.path:
    sys.path.append( path )
else:
    print( f"[{path}] already in sys.path" )

print( sys.path )

import util as du

path = "/var/genie-in-the-box/src"
du.add_to_path( path )

import util_pandas    as up
import util_stopwatch as sw
import util_langchain as ulc
import genie_client   as gc

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

import pandas as pd

# from IPython.display import display, Markdown

debug = True
verbose = True


['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev', '/var/genie-in-the-box/src/notebooks', '/usr/local/lib/python310.zip', '/usr/local/lib/python3.10', '/usr/local/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/site-packages', '/var/genie-in-the-box/src/lib']
Added [/var/genie-in-the-box/src] to sys.path
pwd [/var/genie-in-the-box/src/notebooks]

/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug
/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev
/usr/local/lib/python3.10
/usr/local/lib/python3.10/lib-dynload
/usr/local/lib/python3.10/site-packages
/usr/local/lib/python310.zip
/var/genie-in-the-box/src
/var/genie-in-the-box/src/lib
/var/genie-in-the-box/src/notebooks


In [6]:
pandas_prompt_template = """
You are working with a pandas dataframe in Python. The name of the dataframe is `df`.

This is the ouput from `print(df.head().to_csv())`, in CSV format:

"start_date","end_date","start_time","end_time","event_type","recurrent","recurrence_interval","priority_level","name","relationship","description_who_what_where"
"2023-07-01","2023-07-04","00:00","23:59","Concert",False,"","none","Jenny","coworker","Concert of Jenny at the city center"
"2023-07-01","2023-07-01","05:25","17:22","TODO",False,"","highest","Gregorio","friend","Send out invitations for the party for Gregorio"
"2023-07-01","2023-07-01","13:27","01:59","Appointment",False,"","high","Leroy Ruiz","father","Appointment with Leroy Ruiz at the clinic"
"2023-07-03","2023-07-04","00:00","23:59","Subscription",True,"1 year","highest","Leroy Ruiz","father","Renewal of Leroy Ruiz's subscription"
"2023-07-03","2023-07-04","00:00","23:59","Anniversary",True,"4 week","none","Juan","neighbor","Juan's anniversary celebration at the park"
"2023-07-04","2023-07-04","00:00","23:59","Anniversary",True,"4 week","medium","Leroy Ruiz","father","Leroy Ruiz's anniversary celebration at the park"

This is the from `print(df.dtypes)`:

start_date                    datetime64[ns]
end_date                      datetime64[ns]
start_time                            object
end_time                              object
event_type                            object
recurrent                               bool
recurrence_interval                   object
priority_level                        object
name                                  object
relationship                          object
description_who_what_where            object
dtype: object

As you generate the python code needed to answer the question asked of you below, I want you to:

1) Question: Ask yourself If you understand the question I am asking you?
2) Think: Before you do anything, think out loud about what I'm asking you to do, including listing the steps you need to take to solve this problem. What are the things you need to consider? Be critical of your thought process!
3) Code: Generate a verbatim list of code that you used to arrive at your answer, one line of code per item on the list. The code must be complete, syntactically correct, and run to completion. The last line of your code must be the variable `solution`, which represents the answer.
4) Return: Report on the object type of the variable `solution` in your last line of code.
5) Explain: Briefly and succinctly explain your code in plain English.

Format: return your response as a JSON object with the following fields:
{{
    "question": "the question that your code answers",
    "thoughts": "your thoughts",
    "code": [],
    "return": "Object type of the variable `solution`",
    "explanation": "your brief explanation of your code"
}}

Begin!

Question: {question}
"""

In [7]:
langchain.debug = debug
langchain.verbose = verbose

llm_4 = ChatOpenAI( model_name="gpt-4-0613", temperature=0.0 )
memory = ConversationBufferMemory( memory_key="chat_history", return_messages=True, verbose=verbose )
df = pd.read_csv( du.get_project_root() + "/src/conf/long-term-memory/events.csv" )
df = up.cast_to_datetime( df, debug=debug )

calendar_agent = create_pandas_dataframe_agent( llm_4, df, verbose=verbose )

calendar_prompt = PromptTemplate(
    template=pandas_prompt_template,
    input_variables=[ "question" ]
)
calendar_chain = LLMChain(
    prompt=calendar_prompt,
    llm=llm_4
)
calendar_agent.memory = memory


------------------------------------------------------------------------------------------------------------------------
- df.dtypes:
------------------------------------------------------------------------------------------------------------------------
start_date                    datetime64[ns]
end_date                      datetime64[ns]
start_time                            object
end_time                              object
event_type                            object
recurrent                               bool
recurrence_interval                   object
priority_level                        object
name                                  object
relationship                          object
description_who_what_where            object
dtype: object


In [8]:
df.dtypes

start_date                    datetime64[ns]
end_date                      datetime64[ns]
start_time                            object
end_time                              object
event_type                            object
recurrent                               bool
recurrence_interval                   object
priority_level                        object
name                                  object
relationship                          object
description_who_what_where            object
dtype: object

In [9]:
foo = df.info()
# print( foo )
foo

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   start_date                  200 non-null    datetime64[ns]
 1   end_date                    200 non-null    datetime64[ns]
 2   start_time                  200 non-null    object        
 3   end_time                    200 non-null    object        
 4   event_type                  200 non-null    object        
 5   recurrent                   200 non-null    bool          
 6   recurrence_interval         47 non-null     object        
 7   priority_level              200 non-null    object        
 8   name                        200 non-null    object        
 9   relationship                200 non-null    object        
 10  description_who_what_where  200 non-null    object        
dtypes: bool(1), datetime64[ns](2), object(8)
memory usage: 15.

In [10]:
def get_question_and_coda( event_query ):
    
    question_plus_coda_template = """
    Question: {event_query}  
    
    Hint: An event that I have today may have started before today and may end tomorrow or next week, so be careful how you filter on dates.
    Hint: When filtering by dates, use `pd.Timestamp( day )` to convert a Python datetime object into a Pandas `datetime64[ns]` value.
    Hint: If your code returns any records, they should include all columns in the dataframe.
    """
    
    return question_plus_coda_template.format( event_query=event_query )


event_query = "What events do I have today?"
question = get_question_and_coda( event_query )

for line in question.split( "\n" ):
    print( line )



    Question: What events do I have today?  
    
    Hint: An event that I have today may have started before today and may end tomorrow or next week, so be careful how you filter on dates.
    Hint: When filtering by dates, use `pd.Timestamp( day )` to convert a Python datetime object into a Pandas `datetime64[ns]` value.
    Hint: If your code returns any records, they should include all columns in the dataframe.
    


In [11]:
timer = sw.Stopwatch( "Running pandas prompt..." )
event_query = "What events do I have today?"
question = get_question_and_coda( event_query )
my_response = calendar_chain.run( question=question )
timer.print( "Done!" )

Running pandas prompt...
[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "question": "\n    Question: What events do I have today?  \n    \n    Hint: An event that I have today may have started before today and may end tomorrow or next week, so be careful how you filter on dates.\n    Hint: When filtering by dates, use `pd.Timestamp( day )` to convert a Python datetime object into a Pandas `datetime64[ns]` value.\n    Hint: If your code returns any records, they should include all columns in the dataframe.\n    "
}
[llm/start] [1:chain:LLMChain > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\n\nThis is the ouput from `print(df.head().to_csv())`, in CSV format:\n\n\"start_date\",\"end_date\",\"start_time\",\"end_time\",\"event_type\",\"recurrent\",\"recurrence_interval\",\"priority_level\",\"name\",\"relationship\",\"description_who_what_where\"\n\"20

In [12]:
response_dict = json.loads( my_response )
response_dict

{'question': 'What events do I have today?',
 'thoughts': "To answer this question, I need to filter the dataframe to only include events that are happening today. This means that the event's start date is on or before today and the end date is on or after today. I will use the pandas Timestamp function to convert today's date into a format that can be compared with the start_date and end_date columns in the dataframe. I will then use boolean indexing to filter the dataframe.",
 'code': ['import pandas as pd',
  "today = pd.Timestamp('today').normalize()",
  "solution = df[(df['start_date'] <= today) & (df['end_date'] >= today)]"],
 'return': 'pandas.core.frame.DataFrame',
 'explanation': "The code first imports the pandas library. Then it creates a variable 'today' which holds the current date in a format that can be compared with the dates in the dataframe. The normalize() function is used to set the time part of the Timestamp to midnight, as we are only interested in the date. The l

In [13]:
# response_dict[ "code" ] = response_dict[ "code" ][ 0:4 ]
# # response_dict[ "code" ].append( "solution = df_today" )
for line in response_dict[ "code" ]:
    print( line )


import pandas as pd
today = pd.Timestamp('today').normalize()
solution = df[(df['start_date'] <= today) & (df['end_date'] >= today)]


In [14]:
%autoreload
# Let's copy it until we get the code munging right
response_dict_copy = response_dict[ "code" ][ : ]

response = ulc.assemble_and_run_solution(
    response_dict_copy, path="/src/conf/long-term-memory/events.csv",
    solution_code_returns=response_dict[ "return" ], debug=False
)
print()
for line in response[ "response" ].split( "\n" ):
    print( line )


{"start_date":1695254400000,"end_date":1695859200000,"start_time":"00:00","end_time":"23:59","event_type":"Concert","recurrent":false,"recurrence_interval":null,"priority_level":"high","name":"Juan","relationship":"neighbor","description_who_what_where":"Concert of Juan at the city center"}
{"start_date":1695340800000,"end_date":1695945600000,"start_time":"00:00","end_time":"23:59","event_type":"Conference","recurrent":false,"recurrence_interval":null,"priority_level":"none","name":"Barbara Jane Ruiz","relationship":"mother","description_who_what_where":"Conference with Barbara Jane Ruiz on AI advancements"}
{"start_date":1695513600000,"end_date":1696032000000,"start_time":"12:01","end_time":"04:55","event_type":"TODO","recurrent":false,"recurrence_interval":null,"priority_level":"medium","name":"Juan","relationship":"neighbor","description_who_what_where":"Prepare presentation for next week's meeting for Juan (neighbor)"}
{"start_date":1695600000000,"end_date":1696118400000,"start_ti

In [15]:
len( response[ "response" ].split( "\n" ) )

10

In [16]:
rows = len( response[ "response" ].split( "\n" ) )
preamble = f"""
I'm going to show you {rows} rows of JSONL output from a query on a pandas dataframe. 

JSONL output:

{response[ "response" ]}
"""
#for line in preamble:
#     print( line )
print( preamble )


I'm going to show you 10 rows of JSONL output from a query on a pandas dataframe. 

JSONL output:

{"start_date":1695254400000,"end_date":1695859200000,"start_time":"00:00","end_time":"23:59","event_type":"Concert","recurrent":false,"recurrence_interval":null,"priority_level":"high","name":"Juan","relationship":"neighbor","description_who_what_where":"Concert of Juan at the city center"}
{"start_date":1695340800000,"end_date":1695945600000,"start_time":"00:00","end_time":"23:59","event_type":"Conference","recurrent":false,"recurrence_interval":null,"priority_level":"none","name":"Barbara Jane Ruiz","relationship":"mother","description_who_what_where":"Conference with Barbara Jane Ruiz on AI advancements"}
{"start_date":1695513600000,"end_date":1696032000000,"start_time":"12:01","end_time":"04:55","event_type":"TODO","recurrent":false,"recurrence_interval":null,"priority_level":"medium","name":"Juan","relationship":"neighbor","description_who_what_where":"Prepare presentation for next 

In [17]:
instructions = f"""
Reformat and rephrase the JSONL data above in conversational English so that it answers this question: `{event_query}`

Your output should contain one line per event.
"""
for line in instructions.split( "\n" ):
    print( line )


Reformat and rephrase the JSONL data above in conversational English so that it answers this question: `What events do I have today?`

Your output should contain one line per event.


In [18]:
%autoreload
genie_client = gc.GenieClient()
answer_conversational = genie_client.ask_chat_gpt_text( instructions, preamble=preamble, model=gc.GPT_4 )

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5180:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5180:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5180:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5703:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2666:(snd_pcm_open_noupdate) Unknown PCM sysdefault
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5180:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5180:(_snd_config_evaluate) function snd_func_concat returned error: No

Asking ChatGPT [gpt-4-0613]...Done! in 29,840 ms


In [19]:
for line in answer_conversational.split( "\n" ):
    print( line )

Assuming today's date corresponds to the start_date in the JSONL data, here are your events:

1. You have a concert featuring Juan at the city center. It's a high-priority event that lasts the whole day.
2. There's a conference with your mother, Barbara Jane Ruiz, on AI advancements. This event doesn't have a priority level and also lasts the entire day.
3. You need to prepare a presentation for next week's meeting for your neighbor, Juan. This task is of medium priority and starts at 12:01 and ends at 04:55.
4. You have another meeting with your sister, Sally. It's a low-priority event that starts at 21:58 and ends at 00:54.
5. There's a conference with your father, Leroy Ruiz, on AI advancements. This is your highest priority event and it lasts the entire day.
6. It's your neighbor Juan's birthday party at their favorite bar. This is a recurring event every 2 months and it lasts the whole day.
7. It's also your friend Gregorio's birthday party at their favorite bar. This is a recurri

In [23]:
genie_client = gc.GenieClient()
answer_conversational = genie_client.ask_chat_gpt_text( instructions, preamble=preamble, model=gc.GPT_3_5 )

Asking ChatGPT [gpt-3.5-turbo-0613]...Done! in 8,810 ms


In [24]:
import regex as re

In [25]:
# https://chat.openai.com/share/eca4c68e-73b1-4c5c-81c2-151e2216442f
# The very last interaction contains these instructions:
#
# Use the regular expression pattern ^\d+\.\s*:
# 
#     ^: asserts position at the start of a line.
#     \d+: matches one or more digits.
#     \.: matches a literal period (dot).
#     \s*: matches zero or more whitespace characters.

for line in answer_conversational.split( "\n" ):
    line = re.sub( r"^\d+\.\s*", "", line )
    print( f"[{line}]" )

[Today, you have the following events:]
[]
[Event Type: Concert]
[   - Start Date: [Date]]
[   - Start Time: 00:00]
[   - End Time: 23:59]
[   - Description: Concert of Juan at the city center]
[]
[Event Type: Conference]
[   - Start Date: [Date]]
[   - Start Time: 00:00]
[   - End Time: 23:59]
[   - Description: Conference with Barbara Jane Ruiz on AI advancements]
[]
[Event Type: TODO]
[   - Start Date: [Date]]
[   - Start Time: 12:01]
[   - End Time: 04:55]
[   - Description: Prepare presentation for next week's meeting for Juan (neighbor)]
[]
[Event Type: Meeting]
[   - Start Date: [Date]]
[   - Start Time: 21:58]
[   - End Time: 00:54]
[   - Description: Another boring meeting with Sally]
[]
[Please note that the [Date] should be replaced with the actual date.]


# Prototyping the calendar agent object

In [26]:
import openai

In [125]:
GPT_4   = "gpt-4-0613"
GPT_3_5 = "gpt-3.5-turbo-0613"

def ask_chat_gpt_text( preamble, query, model=GPT_4 ):

    openai.api_key = os.getenv( "FALSE_POSITIVE_API_KEY" )
    
    timer = sw.Stopwatch( msg=f"Asking ChatGPT [{model}]...".format( model ) )
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[ 
            { "role": "system", "content": preamble },
            { "role": "user",   "content": query } 
        ],
        temperature=0,
        max_tokens=2400,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    timer.print( use_millis=True )
    
    return response[ "choices" ][ 0 ][ "message" ][ "content" ].strip()

preamble = "You are chatGPT, a helpful chatbot"
question = "What's your name?"

ask_chat_gpt_text( preamble, question, model=GPT_4 )

Asking ChatGPT [gpt-4-0613]...
Asking ChatGPT [gpt-4-0613]... in 796 ms


"My name is OpenAI's GPT-3, but you can call me ChatGPT."

In [115]:
%autoreload

import os
import json

import lib.util           as du
import lib.util_pandas    as dup
import lib.util_stopwatch as sw
import genie_client       as gc

import pandas as pd

class CalendaringAgent:
    
    def __init__( self, path_to_df, debug=False, verbose=False ):
        
        self.debug                = debug
        self.verbose              = verbose
        self.path_to_df           = du.get_project_root() + path_to_df
        self.df                   = pd.read_csv( self.path_to_df )
        self.df                   = dup.cast_to_datetime( self.df )
    
        self.pandas_system_prompt = self._get_pandas_system_prompt()
        
        self.response_dict        = None 
        self.question             = None
        
    def _get_pandas_system_prompt( self ):
        
        csv = self.df.head( 3 ).to_csv( header=True, index=False)
        csv = csv + self.df.tail( 3 ).to_csv( header=False, index=False )
        
        pandas_system_prompt = f"""
        You are an expert software engineer working with a pandas dataframe in Python containing calendaring and events information. The name of the dataframe is `df`.

        This is the ouput from `print(df.head().to_csv())`, in CSV format:

        {csv}
        
        This is the output from `print(self.df.event_type.value_counts())`:

        {self.df.event_type.value_counts()}

        As you generate the python code needed to answer the events and calendaring question asked of you below, I want you to:

        1) Question: Ask yourself if you understand the question that I am asking you.  Pay attention to the details!
        2) Think: Before you do anything, think out loud about what I'm asking you to do, including what are the steps that you will need to take to solve this problem. Be critical of your thought process!
        3) Code: Generate a verbatim list of code that you used to arrive at your answer, one line of code per item on the list. The code must be complete, syntactically correct, and capable of runnning to completion. The last line of your code must be the variable `solution`, which represents the answer. Make sure that any filtering you perform matches the question asked of you by the user!
        4) Return: Report on the object type of the variable `solution` in your last line of code. Use one word to represent the object type.
        5) Explain: Briefly and succinctly explain your code in plain English.
        
        Format: return your response as a JSON object in the following fields:
        {{
            "question": "The question, verbatim and without modification, that your code attempts to answer",
            "thoughts": "Your thoughts",
            "code": [],
            "return": "Object type of the variable `solution`",
            "explanation": "A brief explanation of your code",
        }}

        Hint: An event that I have today may have started before today and may end tomorrow or next week, so be careful how you filter on dates.
        Hint: When filtering by dates, use `pd.Timestamp( day )` to convert a Python datetime object into a Pandas `datetime64[ns]` value.
        Hint: If your solution variable is a dataframe, it should include all columns in the dataframe.
        
        The question follows shortly, please wait to answer it until you see it.
        """
        if self.debug: print( pandas_system_prompt )
        
        return pandas_system_prompt
    
    def run_prompt( self, question ):
        
        # timer = sw.Stopwatch( "Running pandas prompt..." )
        self.question      = question
        self.response      = ask_chat_gpt_text( self.pandas_system_prompt, self.question, model=GPT_4 )
        self.response_dict = json.loads( self.response )
        # timer.print( "Done!" )
        
        if self.debug: print( json.dumps( self.response_dict, indent=4 ) )
        
        return self.response_dict
        
    def run_code( self ):
        
        # Let's copy it until we get the code munging right
        response_dict_copy = self.response_dict[ "code" ][ : ]
        
        self.code_response = ulc.assemble_and_run_solution(
            response_dict_copy, path="/src/conf/long-term-memory/events.csv",
            solution_code_returns=response_dict[ "return" ], debug=False
        )
        if self.debug and self.verbose:
            du.print_banner( "Code output", prepend_nl=True )
            for line in self.code_response[ "output" ].split( "\n" ):
                print( line )
                
        return self.code_response     
    
    def format_output( self ):
        
        preamble     = self.get_formatting_preamble()
        instructions = self.get_formatting_instructions()
        
        self.formatted_output = ask_chat_gpt_text( preamble, instructions, model=GPT_3_5 )
        
        return self.formatted_output

    def get_formatting_preamble( self ):
        
        rows        = self.code_response[ "output" ].split( "\n" )
        row_count   = len( rows )
        
        lines       = []
        line_number = 1
        
        for row in rows:
            
            lines.append( f"{line_number}) {row}" )
            line_number += 1
            
        lines = "\n".join( lines )
        
        preamble = f"""
        You are an expert in converting raw data into conversational English.
        
        The following {row_count} rows of JSONL formatted data are the output from a query on a pandas dataframe about events on my calendar. 
        
        JSONL output:
        
        {lines}
        """
        return preamble
    
    def get_formatting_instructions( self ):
        
        instructions = f"""
        Reformat and rephrase the JSONL data that I just showed you in conversational English so that it answers this question: `{self.question}`
        
        Use this format: "You have a two hour lunch date with your friend Bob at noon today at Burgerland.
        
        Each line of the output that you create should contain one event.
        If the query returned zero rows of JSONL data, then say "You have no EVENT_TYPE events today/tomorrow/this week."
        """
        return instructions

In [116]:
agent         = CalendaringAgent( path_to_df="/src/conf/long-term-memory/events.csv", debug=True )

response_dict = agent.run_prompt( "What todo items do I have on my calendar for this week?" )
# print( response_dict )


        You are an expert software engineer working with a pandas dataframe in Python containing calendaring and events information. The name of the dataframe is `df`.

        This is the ouput from `print(df.head().to_csv())`, in CSV format:

        start_date,end_date,start_time,end_time,event_type,recurrent,recurrence_interval,priority_level,name,relationship,description_who_what_where
2023-08-01,2023-08-04,00:00,23:59,Concert,False,,none,Jenny,coworker,Concert of Jenny at the city center
2023-08-01,2023-08-01,05:25,17:22,TODO,False,,highest,Gregorio,friend,Send out invitations for the party for Gregorio
2023-08-01,2023-08-01,13:27,01:59,Appointment,False,,high,Leroy Ruiz,father,Appointment with Leroy Ruiz at the clinic
2023-10-01,2023-10-01,00:00,23:59,Birthday,True,3 day,low,Bob,brother,Bob's birthday party at their favorite bar
2023-10-01,2023-10-01,00:00,23:59,Anniversary,True,3 week,highest,Tom Ruiz,brother,Tom Ruiz's anniversary celebration at the park
2023-10-01,2023-10-04

In [126]:
%autoreload
code_response = agent.run_code()
code_response[ "output" ]

'{"start_date":1695513600000,"end_date":1696032000000,"start_time":"12:01","end_time":"04:55","event_type":"TODO","recurrent":false,"recurrence_interval":null,"priority_level":"medium","name":"Juan","relationship":"neighbor","description_who_what_where":"Prepare presentation for next week\'s meeting for Juan (neighbor)"}\n{"start_date":1695772800000,"end_date":1695772800000,"start_time":"18:55","end_time":"05:58","event_type":"TODO","recurrent":false,"recurrence_interval":null,"priority_level":"low","name":"Pablo","relationship":"friend","description_who_what_where":"Prepare presentation for next week\'s meeting for Pablo"}\n{"start_date":1695945600000,"end_date":1696464000000,"start_time":"16:58","end_time":"13:49","event_type":"TODO","recurrent":false,"recurrence_interval":null,"priority_level":"medium","name":"Juan","relationship":"neighbor","description_who_what_where":"Renew gym membership for Juan (neighbor)"}'

In [133]:
%autoreload
for line in agent.get_formatting_preamble().split( "\n" ):
    print( line )


        You are an expert in converting raw data into conversational English.
        
        The following 3 rows of JSONL formatted data are the output from a query on a pandas dataframe about events on my calendar. 
        
        JSONL output:
        
        1) {"start_date":1695513600000,"end_date":1696032000000,"start_time":"12:01","end_time":"04:55","event_type":"TODO","recurrent":false,"recurrence_interval":null,"priority_level":"medium","name":"Juan","relationship":"neighbor","description_who_what_where":"Prepare presentation for next week's meeting for Juan (neighbor)"}
2) {"start_date":1695772800000,"end_date":1695772800000,"start_time":"18:55","end_time":"05:58","event_type":"TODO","recurrent":false,"recurrence_interval":null,"priority_level":"low","name":"Pablo","relationship":"friend","description_who_what_where":"Prepare presentation for next week's meeting for Pablo"}
3) {"start_date":1695945600000,"end_date":1696464000000,"start_time":"16:58","end_time":"13:49","

In [134]:
for line in agent.get_formatting_instructions().split( "\n" ):
    print( line )


        Reformat and rephrase the JSONL data that I just showed you in conversational English so that it answers this question: `What todo items do I have on my calendar for this week?`
        
        Use this format: "You have a two hour lunch date with your friend Bob at noon today at Burgerland.
        
        Each line of the output that you create should contain one event.
        If the query returned zero rows of JSONL data, then say "You have no EVENT_TYPE events today/tomorrow/this week."
        


In [131]:
agent.format_output()

Asking ChatGPT [gpt-3.5-turbo-0613]...
Asking ChatGPT [gpt-3.5-turbo-0613]... in 7,206 ms


"You have a medium priority TODO item on your calendar for this week. It is to prepare a presentation for next week's meeting for your neighbor Juan. The event starts on Monday, January 24th at 12:01 PM and ends on Friday, January 28th at 4:55 AM.\n\nYou have a low priority TODO item on your calendar for this week. It is to prepare a presentation for next week's meeting for your friend Pablo. The event starts and ends on Tuesday, January 25th. It starts at 6:55 PM and ends at 5:58 AM.\n\nYou have a medium priority TODO item on your calendar for this week. It is to renew your gym membership for your neighbor Juan. The event starts on Thursday, January 27th at 4:58 PM and ends on Monday, January 31st at 1:49 PM."

In [132]:
for line in agent.formatted_output.split( "\n" ): 
    print( line )

You have a medium priority TODO item on your calendar for this week. It is to prepare a presentation for next week's meeting for your neighbor Juan. The event starts on Monday, January 24th at 12:01 PM and ends on Friday, January 28th at 4:55 AM.

You have a low priority TODO item on your calendar for this week. It is to prepare a presentation for next week's meeting for your friend Pablo. The event starts and ends on Tuesday, January 25th. It starts at 6:55 PM and ends at 5:58 AM.

You have a medium priority TODO item on your calendar for this week. It is to renew your gym membership for your neighbor Juan. The event starts on Thursday, January 27th at 4:58 PM and ends on Monday, January 31st at 1:49 PM.
